In [1]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="../.envfiles/dev.env")

BQ_HOST = os.environ["BQ_HOST"]
BQ_PORT = os.environ["BQ_PORT"]
BQ_PROJECT = os.environ["BQ_PROJECT"]
BQ_DATASET = os.environ["BQ_DATASET"]

Create ClientOptions with api_endpoint option and use AnonymousCredentials to disable authentication.

In [2]:
from google.api_core.client_options import ClientOptions
from google.auth.credentials import AnonymousCredentials
from google.cloud import bigquery
from google.cloud.bigquery import QueryJobConfig

client_options = ClientOptions(api_endpoint=f"http://{BQ_HOST}:{BQ_PORT}")
client = bigquery.Client(
  project=BQ_PROJECT,
  client_options=client_options,
  credentials=AnonymousCredentials(),
)
dataset_id = f"{client.project}.{BQ_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset skill-master.dev


In [3]:
# Create tabel
schema = [
    bigquery.SchemaField("full_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("age", "INTEGER", mode="REQUIRED"),
]
table_id = f"{dataset_id}.people"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table skill-master.dev.people


In [4]:
# Insert your query here
rows_to_insert = [
    {"full_name": "Phred Phlyntstone", "age": 32},
    {"full_name": "Wylma Phlyntstone", "age": 29},
]

errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

New rows have been added.


In [ ]:
query = f"""
    SELECT full_name, age 
    FROM {BQ_PROJECT}.{BQ_DATASET}.people
"""
rows = client.query_and_wait(query)  # Make an API request.

print("The query data:")
for row in rows:
    # Row values can be accessed by field name or index.
    print("name={}, age={}".format(row[0], row["age"]))

KeyboardInterrupt: 